In [1]:

import cv2
import numpy as np
import pandas as pd
# from PIL import Image as im
# from IPython import display
from tkinter import *
from tkinter import filedialog , PhotoImage 
import os
import cv2
import tkinter as tk
from PIL import Image as im
from PIL import Image, ImageTk

In [2]:
classeFile = 'CocoNames.txt'
classesNames=[]
with open(classeFile , 'rt') as f :
    classesNames = f.read().rstrip('\n').split('\n')

In [3]:
modelConfig = 'yolov3.cfg'
modelWeights = 'yolov3.weights'
# create Network
net = cv2.dnn.readNetFromDarknet(modelConfig , modelWeights)
# use Opencv as the backend , and CPU
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
def giveOutputsLayerDetails(path):
    image = cv2.imread(path)
    blob = cv2.dnn.blobFromImage(image , 1/255 , (320 , 320) , [0,0,0] , 1 , crop = False)# facteur de mesure appliqué sur les pixels de l'image pour les normaliser 
    net.setInput(blob)# 1 : le facteur de mesure après la  normalisation
    LayersNames =net.getLayerNames()
    IndexOutLayers = np.array(net.getUnconnectedOutLayers()).reshape(-1 , 1) 
    outLayersNames = [LayersNames[i[0]-1] for i in IndexOutLayers]
    outputs = net.forward(outLayersNames)
    return outputs 

In [5]:
# ----------------------------------------------------------
def findObject(outputs , imgMat) :
    v_conf = 0.8
    hT , wT , cT = imgMat.shape
    bbox = []
    classesIds = []
    confs = []
    
    for i in outputs :
        for j in i :
            scores = j[5:] 
            classId = np.argmax(scores) 
            confidence = scores[classId]
            if confidence > v_conf :
                w , h = int(j[2]*wT) , int(j[3]*hT) 
                x , y = int((j[0]*wT)-w/2) , int((j[1]*hT) -h/2) 
                bbox.append([x , y , w , h])
                classesIds.append(classId)
                confs.append(float(confidence))
    
    return bbox , classesIds , confs

In [6]:
def NMS_Draw(box , classes , probConf , imgPath):# , Save_Path , NameChoose ):
    imgMat = cv2.imread(imgPath)
    indicesKeeping = cv2.dnn.NMSBoxes(box , probConf , score_threshold = 0.8 , nms_threshold = 0.3 )
#     imgPath = cv2.imread(imgPath)    
#     save_image_detect_path = defaultPath(imgPath)
    for i in indicesKeeping :
        TrueBbox = box[i]
        x , y , w , h =TrueBbox[:] 
        cv2.rectangle(imgMat , (x , y) , (x+w , y+h) , (0, 177, 123) , 3)
        cv2.putText(imgMat , " {} {}%".format(classesNames[classes[i]] ,int(probConf[i]*100)) , (x , y-10),
                    cv2.FONT_HERSHEY_SIMPLEX , 0.6 ,
                            (1, 0, 0) , 2)
    Imgdata = im.fromarray(imgMat) # return image format
    return Imgdata
#     saving the image
#     Imgdata.save(save_image_detect_path)

In [7]:
def defaultPath(img_path):
#     print("avant : " , path)
    c = img_path.split("/")
#     print("apres split fonc : ", c)
    SeparerMot = c[-1].split('.')
    NewName = SeparerMot[0] + "detecter"+"."+SeparerMot[-1]
    c.remove(c[-1])
    autoSavePath = "/".join(c) + "/"+ NewName
    return autoSavePath

In [8]:
def detect_images():    
    def detect():
        box , classes , probConf = findObject(giveOutputsLayerDetails(path) , cv2.imread(path))[:]
        imegedetecter = NMS_Draw(box , classes , probConf , path)
        # saving the image
        imegedetecter.save(defaultPath(path))
        pathSave = defaultPath(path)
        imgDetec=Image.open(pathSave,'r')
        imgDetec.thumbnail((350,350)) # thumbnail() is a function that does the thumbnail of a mage given ( reduce the sice of image )
        imgDetec = ImageTk.PhotoImage(imgDetec) # ImageTk module contains support to create and modify 
        lblDetec.configure(image = imgDetec)
        lblDetec.image=imgDetec

        frameS_d = Frame(frameImgDetect , bg = 'purple' )
        frameS_d.place( x=70 , width=700 , height=500)
        frameS_d.pack(side=BOTTOM)
#         button icons : save delete -----------------
#         icSav = PhotoImage(file = './icons/download2.gif')
#         saveicon = icSav.subsample(15 , 15)
        saveBtn = Button(frameS_d , text='save' ,font=("Arial" , 13 , 'bold' ),fg='white',bg='green' , cursor='hand2', command = enregistrer)
        saveBtn.pack(side=RIGHT , padx=1 , pady=1)
#         icDel = PhotoImage(file = 'icons/remove.gif')
#         deleteicon = icDel.subsample(15 , 15)
        deleteBtn = Button(frameS_d , text='delete' ,font=("Arial" , 13 , 'bold' ),fg='white',bg='red', cursor='hand2' , command = deleteImage)
        deleteBtn.pack(side = LEFT , padx=1, pady=2)


    # _________________________________________________________________________
    def showImage():
        global path 
        path=filedialog.askopenfilename(initialdir = "F:/dell/Desktop/PFE/imagePath",filetypes=(("PNG file","*.png"),("JPG File","*.jpg"),("All Files","*.*"),("JPG File","*.jpeg")))
        savepath = defaultPath(path)
        print(f'image path : {path} \nAuto path :{savepath}')
        img=Image.open(path,'r')
        img.thumbnail((350,350))
        img=ImageTk.PhotoImage(img)
        lbl.configure(image=img)
        lbl.image=img
    
    def enregistrer():
        global TruthPathSave
        print(defaultPath(path))
        os.remove(defaultPath(path))
        dirictory = filedialog.askdirectory(title="Sélectionnez un répertoire de destination" , mustexist = True)
        print('you choisir : ' , dirictory)
#         Sélectionnez un répertoire de destination
        Selectnom = tk.simpledialog.askstring(Container , "choose name to image : " )
        print("name chosed " , Selectnom)
        while(Selectnom == "" or Selectnom.isspace()):
            Selectnom = tk.simpledialog.askstring(Container , "choose name to image *the name must be string*: " )
#         print(Selectnom)
        TruthPathSave = dirictory+"/"+Selectnom+".png"
#         pathSave = TruthPathSave
#         print("____" , pathOfimageSave)
        box , classes , probConf = findObject(giveOutputsLayerDetails(path) , cv2.imread(path))[:]
        imgdetec = NMS_Draw(box , classes , probConf , path)
        imgdetec = imgdetec.save(TruthPathSave)
        tk.messagebox.showinfo(title = 'information' , message = "image saved" , parent=Container)
        

    def deleteImage():
#         os.remove(defaultPath(path)) 
        reponse = tk.messagebox.askyesnocancel('your reponse' , " Are you want to delete image ? ")
#         print(rep)   
        if(reponse == True):
            os.remove(TruthPathSave)
            tk.messagebox.showinfo(title="confirmation" , message = "image deleted")
            print('image deleted ')
            
    def Quit_windows():
        root.destroy()

    # _________________________________________________________________________
    root=Tk()
    titre = Label(root , text="Image Detection" , font=("Arial" , 17 , 'bold'),fg='white' , bg='black')
    titre.pack(fill=X)  
    quiticon = PhotoImage(file = 'F:/dell/Desktop/PFE/icons/quit.png')
#  
    frm= Frame(root , bg="#0C2864")
    frm.place(x=100 , y=400 , width=70, height=70)
    frm.pack(side= BOTTOM, padx=50, pady=50)
    quitterbtn = Button(frm , text = "Quitter  " ,image=quiticon,compound=LEFT ,bg='red',font =('Arial',15), cursor="hand2" , fg='white' , command=Quit_windows)
    quitterbtn.pack(side=BOTTOM,padx=3)
    # _________________________________________________________________________
    # ---------  frame 1 : container ------
    global Container
    Container = Frame(root , bg='#264d84')
    Container.place(x=100 , y=50 , width=810, height=500)
    # _________________________________________________________________________
    #   ---------  frame 2 :image importer ------
    #     global frameImg
    frameImg = Frame(Container , bg='white')
    frameImg.place(x=20 , y=30 , width=370, height=430) 
    texte1 = Label(frameImg , text=' Image Import ', font=("Arial" , 16 , 'bold'),fg='white',bg='#0C2864')
    texte1.pack(fill = X )
    lbl = Label(frameImg ,bg="white")
    lbl.pack(pady=100)
    # _________________________________________________________________________
    #   ---------  frame : image detecter  ------
    global frameImgDetect
    frameImgDetect = Frame(root , bg='white')
    frameImgDetect.place(x=520 , y=80 , width=370, height=430) 
    texte2 = Label(frameImgDetect , text=' Image Detect ', font=("Arial" , 16 , 'bold'),fg='white',bg='#0C2864')
    texte2.pack(fill=X )
    lblDetec= Label(frameImgDetect ,bg="white")
    lblDetec.pack(pady=60)
    # _________________________________________________________________________
    # btn1 ____________________________________________
    icon1 = PhotoImage(file = "F:/dell/Desktop/PFE/icons/import.png")
    importIcon = icon1.subsample(15,15) # resizes the image represented by icon1 by reducing its size
    btn1_img=Button(frm, text=" Importer image ", bg='gray',font =('Arial',15), cursor="hand2", image=importIcon,compound =LEFT, command=showImage)#aaff66
    btn1_img.pack(side=BOTTOM, pady=5)
    # btn2_______________________________________________
    icon1 = PhotoImage(file = "F:/dell/Desktop/PFE/icons/detection.png")
    detectionIcon = icon1.subsample(15,15)
    btn2_img= Button(frm,text=" Detecter image " , bg='gray',font =('Arial',15),cursor="hand2" ,image = detectionIcon ,compound=LEFT ,command=detect)
    btn2_img.pack(side=BOTTOM, padx=30)
    # _________________________________________________________________________
    # -------------------------- Windows ----------------------------
    root.title("Image Detection")
    root.geometry("1000x900")
    root.resizable(False , True)
#     root.iconbitmap("./icons/16.png")
    root.configure(bg='#0C2864')#configure() allows you to change the properties of the window.
    root.mainloop()
    

In [ ]:
detect_images()

image path : F:/dell/Desktop/PFE/imagePath/animals.jpeg 
Auto path :F:/dell/Desktop/PFE/imagePath/animalsdetecter.jpeg
F:/dell/Desktop/PFE/imagePath/animalsdetecter.jpeg
you choisir :  
name chosed  None


Exception in Tkinter callback
Traceback (most recent call last):
  File "F:\Nouveau dossier\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_916\1804051413.py", line 49, in enregistrer
    while(Selectnom == "" or Selectnom.isspace()):
AttributeError: 'NoneType' object has no attribute 'isspace'
Exception in Tkinter callback
Traceback (most recent call last):
  File "F:\Nouveau dossier\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_916\1804051413.py", line 66, in deleteImage
    os.remove(TruthPathSave)
NameError: name 'TruthPathSave' is not defined
